In [ ]:
'''
data/              
│
├── train/            
│   ├── A/           
│   │   ├── A_1.jpg
│   │   ├── A_2.jpg
│   │   └── ...
│   │
│   └── B/          
│       ├── B_1.jpg
│       ├── B_2.jpg
│       └── ...
│
└── val/             
    ├── A/
    │   ├── A_1.jpg
    │   ├── A_2.jpg
    │   └── ...
    │
    └── B/
        ├── B_1.jpg
        ├── B_2.jpg
        └── ...

Python: 3.12.3 (main, Jun 18 2025, 17:59:45) [GCC 13.3.0]
OpenCV: 4.12.0
PyTorch: 2.2.2+cu121
Joblib: 1.5.1
NumPy: 1.26.1
Pandas: 2.3.2
Seaborn: 0.13.2
Matplotlib: 3.10.5
Pillow: 10.2.0
Scikit-learn: 1.7.1
Torchvision: 0.17.2+cu121
Facenet-pytorch: 2.6.0

'''

In [ ]:
import os
import cv2
import sys
import time
import glob
import torch
import joblib
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from PIL import Image
from sklearn.svm import SVC
from sklearn import metrics
from sklearn import model_selection
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder, StandardScaler
from torchvision.transforms import Compose, ToTensor, ColorJitter, RandomAffine
from facenet_pytorch import MTCNN, fixed_image_standardization, InceptionResnetV1

warnings.filterwarnings('ignore')
print('warnings ignored!!')

In [ ]:
print("Python:", sys.version)
print("OpenCV:", cv2.__version__)
print("PyTorch:", torch.__version__)
print("Joblib:", joblib.__version__)
print("NumPy:", np.__version__)
print("Pandas:", pd.__version__)
print("Seaborn:", sns.__version__)
import matplotlib
print("Matplotlib:", matplotlib.__version__)
print("Pillow:", Image.__version__)
import sklearn
print("Scikit-learn:", sklearn.__version__)
import torchvision
print("Torchvision:", torchvision.__version__)
import facenet_pytorch
import pkg_resources
version = pkg_resources.get_distribution("facenet-pytorch").version
print("Facenet-pytorch:", version)


In [ ]:
DATA_PATH = '../data/Newdata'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
mtcnn = MTCNN(margin=20, keep_all=False, select_largest=True, post_process=False, device=device)
model = InceptionResnetV1(classify=False, pretrained='casia-webface').to(device).eval()

In [ ]:
data, target = [], []
for usr in os.listdir(DATA_PATH + '/train'):
    for file in glob.glob(os.path.join(DATA_PATH, 'train', usr, '*.*')):
        try:
            img = Image.open(file).convert('RGB')
        except:
            continue

        face = mtcnn(img)
        if face is None:
            continue

        face = fixed_image_standardization(face)  
        face = face.unsqueeze(0).to(device)       

        with torch.no_grad():
            embed = model(face).cpu().numpy()
            data.append(embed[0])
            target.append(usr)

In [ ]:
le = LabelEncoder()
target = le.fit_transform(target)

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(data, target, test_size=0.2, stratify=target, random_state=42)

In [ ]:
pca=PCA()
pca.fit(data)

plt.figure(1, figsize=(12,8))

plt.plot(pca.explained_variance_, linewidth=2)
 
plt.xlabel('Components')
plt.ylabel('Explained Variaces')
plt.show()

In [ ]:
pipe = Pipeline([
    ("scaler", StandardScaler()),
    ('pca', PCA()),
    ("svc", SVC(kernel="rbf", C=1, gamma="scale", probability=True))
])

param_grid = {
    'pca__n_components': [90, 80, 60],
    'svc__kernel': ['linear', 'rbf'],
    'svc__C': [0.1, 1, 10, 100, 1000],
    'svc__gamma': [1, 0.1, 0.01, 0.001, 0.0001]
}

grid = model_selection.GridSearchCV(pipe, param_grid, cv=5, n_jobs=-1)
grid.fit(X_train, y_train)

print('Best params: ', grid.best_params_)
print('Best cross-val score: ', grid.best_score_)

y_pred = grid.predict(X_test)
print('Classification report: \n', metrics.classification_report(y_test, y_pred))

In [ ]:
cm = metrics.confusion_matrix(y_test, y_pred)

labels = le.classes_
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=labels, yticklabels=labels)

plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.title('Confusion Matrix Heatmap')
plt.xticks(rotation=60)
plt.yticks(rotation=0)
plt.show()

In [ ]:
os.makedirs('output', exist_ok=True)
joblib.dump(grid, 'output/out_file_4' + '/clf_model.pkl')
joblib.dump(target, 'output/out_file_4' + '/label_encoder.pkl')

In [ ]:
clf = joblib.load('output/out_file_4/clf_model.pkl')
label = joblib.load('output/out_file_4/label_encoder.pkl')
os.makedirs('out_val', exist_ok=True)

for usr in os.listdir(DATA_PATH + '/val'):
    count = 2
    for file in glob.glob(os.path.join(DATA_PATH, 'val', usr, '*.*')):
        if count == 0:
            break

        try:
            img = Image.open(file).convert('RGB')
        except:
            continue
        
        face = mtcnn(img)
        box, _ = mtcnn.detect(img)

        if box is None:
            continue

        face = fixed_image_standardization(face)  
        face = face.unsqueeze(0).to(device)       

        with torch.no_grad():
            embed = model(face).numpy()

        prediction = clf.predict(embed)
        score = np.max(clf.predict_proba(embed))
        
        bbox = list(map(int, box[0].tolist()))
        img = np.array(img)
        if score > 0.5:
            frame = cv2.rectangle(img, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 255, 0), 2)
            frame = cv2.putText(frame, str(le.inverse_transform(prediction)[0]) + '_{:.2f}'.format(score), (bbox[0], bbox[1] - 5),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2, cv2.LINE_4)
        else:
            frame = cv2.rectangle(img, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 0, 255), 2)
            frame = cv2.putText(frame, 'Unknown', (bbox[0], bbox[1]), cv2.FONT_HERSHEY_SIMPLEX, 0.6,
                                (0, 255, 0), 2, cv2.LINE_4)
        
        plt.imshow(frame)
        plt.show()            
        count -= 1           

In [ ]:
from lazypredict.Supervised import LazyClassifier

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models, predictions = clf.fit(X_train_scaled, X_test_scaled, y_train, y_test)
models